In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
training = spark.read.format('libsvm').load('Dataset/sample_linear_regression_data.txt')

In [5]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [6]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

In [7]:
lrModel = lr.fit(training)

In [8]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [9]:
lrModel.intercept

0.14228558260358093

In [10]:
training_summary = lrModel.summary

In [11]:
training_summary.r2

0.027839179518600154

In [12]:
training_summary.rootMeanSquaredError

10.16309157133015

In [13]:
all_data = spark.read.format('libsvm').load('Dataset/sample_linear_regression_data.txt')

In [14]:
train_data, test_data = all_data.randomSplit([0.7, 0.3])

In [15]:
train_data.describe().show()

+-------+--------------------+
|summary|               label|
+-------+--------------------+
|  count|                 348|
|   mean|-0.17758830477532958|
| stddev|  10.387602001164677|
|    min| -28.571478869743427|
|    max|   27.78383192005107|
+-------+--------------------+



In [16]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                153|
|   mean| 1.2451113070686635|
| stddev| 10.121682854830558|
|    min|-28.046018037776633|
|    max| 27.111027963108548|
+-------+-------------------+



In [17]:
correct_model = lr.fit(train_data)

In [18]:
test_results = correct_model.evaluate(test_data)

In [19]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-26.879084856744242|
|-23.941426186068732|
| -21.24639858575675|
|  -21.6291730157467|
| -16.96688352619398|
| -18.21405859513429|
|-14.056268944308133|
|-13.844580131303807|
|-14.941090010951866|
| -17.73414399215531|
|-15.987520638319728|
|-15.278923754818583|
| -10.51297524595645|
|-10.230941979212108|
|-13.707322271292947|
|-6.3732353561248845|
|-12.145038284409614|
|-10.886553656738545|
| -7.568647502537313|
|-14.401605877088908|
+-------------------+
only showing top 20 rows



In [20]:
test_results.rootMeanSquaredError

10.28639545986083

In [21]:
unlabeled_data = test_data.select('features')

In [22]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [23]:
predictions = correct_model.transform(unlabeled_data)

In [24]:
predictions.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...|-1.1669331810323915|
|(10,[0,1,2,3,4,5,...|-2.7947809965329906|
|(10,[0,1,2,3,4,5,...|-1.5910618311625926|
|(10,[0,1,2,3,4,5,...|0.19678525158089527|
|(10,[0,1,2,3,4,5,...|-0.8367426624705379|
|(10,[0,1,2,3,4,5,...|   1.95262831968156|
|(10,[0,1,2,3,4,5,...| -1.381115849123085|
|(10,[0,1,2,3,4,5,...|-1.4664004581124823|
|(10,[0,1,2,3,4,5,...|0.11893710120067752|
|(10,[0,1,2,3,4,5,...|  3.405165483079868|
|(10,[0,1,2,3,4,5,...| 2.2150790766168558|
|(10,[0,1,2,3,4,5,...|   2.30107502942648|
|(10,[0,1,2,3,4,5,...|-1.9546811350764104|
|(10,[0,1,2,3,4,5,...|-1.6264083862173169|
|(10,[0,1,2,3,4,5,...|  1.880249274900376|
|(10,[0,1,2,3,4,5,...|-5.2673143217639415|
|(10,[0,1,2,3,4,5,...| 0.5292630193939869|
|(10,[0,1,2,3,4,5,...| 0.2864233148295134|
|(10,[0,1,2,3,4,5,...|-3.0090508017059068|
|(10,[0,1,2,3,4,5,...|  4.045687276714227|
+----------